# API 요청 url과 요청변수 지정을 통한 데이터 수집

- 요청페이지 url : https://www.data.go.kr/data/15039250/openapi.do

- 요청변수(parameters)
    - ServiceKey : 인증키
    - pageNo : 페이지 번호
    - numOfRows : 한 페이지 결과 수
    - type : 호출문서 형식

In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 요청 url
url = 'http://apis.data.go.kr/1741000/EmergencyAssemblyArea_Earthquake2/getArea1List'

# 인증키
key = '4ByaILC8stX6EBiqSrp79NuwtO+i7RIDgEVXAQNWkp8u4Y5J+eN3AB7pLETESl6y7R0W5nEZejRPdWbJOcpqmQ=='

# 요청 parameters
params = {'Servicekey':key,
         'pageNo':1,
         'numOfRows':10,
         'type':'xml'}

#### 페이지 요청

In [ ]:
html = requests.get(url, params = params).text

#### html parsing

In [ ]:
soup = BeautifulSoup(html, 'xml')

In [ ]:
soup

<?xml version="1.0" encoding="utf-8"?>
<EarthquakeOutdoorsShelter>
<head>
<totalCount>11109</totalCount>
<numOfRows>10</numOfRows>
<pageNo>1</pageNo>
<type>XML</type>
<RESULT>
<resultCode>INFO-0</resultCode>
<resultMsg>NOMAL SERVICE</resultMsg>
</RESULT>
</head>
<row>
<arcd>4113000000</arcd>
<acmdfclty_sn>123</acmdfclty_sn>
<ctprvn_nm>경기도</ctprvn_nm>
<sgg_nm>성남시</sgg_nm>
<vt_acmdfclty_nm>위례한빛초등학교 운동장</vt_acmdfclty_nm>
<rdnmadr_cd>4113110800103380002000001</rdnmadr_cd>
<bdong_cd>4113110800</bdong_cd>
<hdong_cd>4113162500</hdong_cd>
<dtl_adres>경기도 성남시 수정구 창곡동 570-0</dtl_adres>
<fclty_ar>2660</fclty_ar>
<xcord>127.14697120254982</xcord>
<ycord>37.46556167726204</ycord>
</row>
<row>
<arcd>4113000000</arcd>
<acmdfclty_sn>124</acmdfclty_sn>
<ctprvn_nm>경기도</ctprvn_nm>
<sgg_nm>성남시</sgg_nm>
<vt_acmdfclty_nm>위례중앙중학교 운동장</vt_acmdfclty_nm>
<rdnmadr_cd>4113110800100920001000001</rdnmadr_cd>
<bdong_cd>4113110800</bdong_cd>
<hdong_cd>4113162500</hdong_cd>
<dtl_adres>경기도 성남시 수정구 창곡동 503-0</dtl_adres>


In [ ]:
rests = soup.find_all('row')
len(rests)

10

In [ ]:
rests[0]

<row>
<arcd>4113000000</arcd>
<acmdfclty_sn>123</acmdfclty_sn>
<ctprvn_nm>경기도</ctprvn_nm>
<sgg_nm>성남시</sgg_nm>
<vt_acmdfclty_nm>위례한빛초등학교 운동장</vt_acmdfclty_nm>
<rdnmadr_cd>4113110800103380002000001</rdnmadr_cd>
<bdong_cd>4113110800</bdong_cd>
<hdong_cd>4113162500</hdong_cd>
<dtl_adres>경기도 성남시 수정구 창곡동 570-0</dtl_adres>
<fclty_ar>2660</fclty_ar>
<xcord>127.14697120254982</xcord>
<ycord>37.46556167726204</ycord>
</row>

## 지진옥외대피소 전체 데이터 수집

In [ ]:
# 요청 url
url = 'http://apis.data.go.kr/1741000/EmergencyAssemblyArea_Earthquake2/getArea1List'

# 인증키
key = '4ByaILC8stX6EBiqSrp79NuwtO+i7RIDgEVXAQNWkp8u4Y5J+eN3AB7pLETESl6y7R0W5nEZejRPdWbJOcpqmQ=='

columns = ['arcd', 'acmdfclty_sn', 'ctprvn_nm', 'sgg_nm', 'vt_acmdfclty_nm',
          'rdnmadr_cd', 'bdong_cd', 'hdong_cd', 'dtl_adres', 'fclty_ar',
          'xcord', 'ycord']

rest_list = []

for i in range(1, 12):
    params = {'Servicekey':key,
         'pageNo':str(i),
         'numOfRows':1000,
         'type':'xml'}
    res = requests.get(url, params=params)
    soup = BeautifulSoup(res.text, 'xml')
    items = soup.find_all('row')
    for item in items:
        rest = {}
        for col in columns:
            rest[col] = item.find(col).text
        rest_list.append(rest)
len(rest_list)

11000

In [ ]:
df = pd.DataFrame(rest_list, columns = columns)
df.head(3)

,arcd,acmdfclty_sn,ctprvn_nm,sgg_nm,vt_acmdfclty_nm,rdnmadr_cd,bdong_cd,hdong_cd,dtl_adres,fclty_ar,xcord,ycord
0,4113000000,123,경기도,성남시,위례한빛초등학교 운동장,4113110800103380002000001,4113110800,4113162500,경기도 성남시 수정구 창곡동 570-0,2660,127.14697120254982,37.46556167726204
1,4113000000,124,경기도,성남시,위례중앙중학교 운동장,4113110800100920001000001,4113110800,4113162500,경기도 성남시 수정구 창곡동 503-0,3286,127.13811475397762,37.47172372636054
2,4113000000,92,경기도,성남시,성남매송초등학교 운동장,4113510600101360000055590,4113510600,4113561000,경기도 성남시 분당구 이매동 136-0,8779,127.12283938580048,37.40169445095525


In [ ]:
df.tail(3)

,arcd,acmdfclty_sn,ctprvn_nm,sgg_nm,vt_acmdfclty_nm,rdnmadr_cd,bdong_cd,hdong_cd,dtl_adres,fclty_ar,xcord,ycord
10997,4817000000,32,경상남도,진주시,진주기계공업고등학교 운동장,4817012100102720001000006,4817012100,4817068000,경상남도 진주시 상평동 272-1,14450,128.10324800426073,35.17711908714348
10998,4817000000,56,경상남도,진주시,가람초등학교 운동장,4817012100102780001096826,4817012100,4817068000,경상남도 진주시 상평동 278-1,2767,128.10156780744964,35.173337663501385
10999,4817000000,36,경상남도,진주시,장재초등학교 운동장,4817012200109280000000002,4817012200,4817069500,경상남도 진주시 초전동 1691-0,2257,128.11015884281957,35.214401020613465


In [ ]:
# 필요없는 열 삭제(지역코드,시설일련번호)
df = df.drop(['arcd', 'acmdfclty_sn'], axis=1)
df.head(2)

,ctprvn_nm,sgg_nm,vt_acmdfclty_nm,rdnmadr_cd,bdong_cd,hdong_cd,dtl_adres,fclty_ar,xcord,ycord
0,경기도,성남시,위례한빛초등학교 운동장,4113110800103380002000001,4113110800,4113162500,경기도 성남시 수정구 창곡동 570-0,2660,127.14697120254982,37.46556167726204
1,경기도,성남시,위례중앙중학교 운동장,4113110800100920001000001,4113110800,4113162500,경기도 성남시 수정구 창곡동 503-0,3286,127.13811475397762,37.47172372636054


In [ ]:
# 데이터프레임 csv 파일로 저장하기
df.to_csv("전국 지진옥외대피소.csv")